# IC - Rodrigo Gossi: MachineLearning aplicado a predição da área de fungos em placas de petri a partir de visão computadional


## Sobre os dados utilizados



## Imports de bibliotecas

In [5]:
!pip install -U julearn
!pip install matplotlib
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.7 MB/s eta 0:00:000:00:01


In [3]:
import numpy as np
import pandas as pd
import sklearn
import julearn
from julearn import run_cross_validation
from julearn.model_selection import RepeatedContinuousStratifiedKFold
import matplotlib.pyplot as plt

In [8]:
!pip show julearn

Name: julearn
Version: 0.3.4
Summary: Juelich Machine Learning Library
Home-page: 
Author: 
Author-email: Fede Raimondo <f.raimondo@fz-juelich.de>, Sami Hamdan <s.hamdan@fz-juelich.de>
License: AGPL-3.0-only
Location: /home/runner/workspace/.pythonlibs/lib/python3.12/site-packages
Requires: numpy, pandas, scikit-learn, statsmodels
Required-by: 


In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive/')

## Imports dos dados necessários

In [4]:
dfA = pd.read_excel("./data/area.xlsx")
dfC = pd.read_excel("./data/caracteristicas.xlsx")

# Remover colunas indesejadas do DataFrame dfC
dfC_clean = dfC.drop(columns=['attributes', 'samples', 'dimension'])

In [45]:
dfA

,identificador,Area(mm²),Diametro médio
0,d1_t2_20uL_r1,58.971,0.866511
1,d1_t2_20uL_r2,59.818,0.872712
2,d1_t2_20uL_r3,62.882,0.894784
3,d1_t2_20uL_r4,57.109,0.852722
4,d1_t2_20uL_r5,99.768,1.127069
...,...,...,...
537,d5_t6_0uL_r3,5619.967,8.459057
538,d5_t6_0uL_r4,5743.332,8.551396
539,d5_t6_0uL_r5,5762.667,8.565778
540,d5_t6_0uL_r6,5472.916,8.347654


In [46]:
dfC

,identificador,attributes,samples,dimension,cor_RG,cor_RB,cor_RH,cor_RS,cor_RV,cor_RI,...,com_inertia,com_energy,com_correlation,com_homogeniety,FFT_energy,FFT_entropy,FFT_intertia,FFT_homogeneity,SNM,EME
0,d1_t2_100ul_r1,7,12198720,1.742674e+06,0.995674,0.989471,-0.383639,-0.130794,0.997174,0.997820,...,30.267896,0.001518,0.995921,0.535464,0.882896,-0.039847,9.580998,0.001363,0.147521,4.186365
1,d1_t2_100uL_r2,7,12198720,1.742674e+06,0.994981,0.989579,0.075174,0.118834,0.996522,0.997343,...,13.167428,0.003691,0.996867,0.516007,0.781416,-0.061916,31.988921,0.000541,0.001729,8.132289
2,d1_t2_100uL_r3,7,12198720,1.742674e+06,0.995855,0.990799,0.032431,0.017351,0.996876,0.997784,...,12.198123,0.003542,0.997089,0.519729,0.764936,-0.061312,31.311636,0.000535,0.001321,8.409600
3,d1_t2_100uL_r3B,7,12198720,1.742674e+06,0.997074,0.991002,-0.238737,-0.181805,0.997676,0.998344,...,19.078551,0.001026,0.997699,0.396827,0.842077,-0.060000,26.460279,0.000504,0.048926,6.696373
4,d1_t2_100uL_r4,7,12198720,1.742674e+06,0.996877,0.990841,-0.418375,-0.322490,0.998054,0.998266,...,26.928049,0.001044,0.996857,0.392886,0.820254,-0.064946,30.044119,0.000526,0.069647,8.170084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,d5_t6_0uL_r3,7,12198720,1.742674e+06,0.999056,0.987996,-0.300870,-0.671761,0.999079,0.999271,...,4.164741,0.003521,0.999539,0.602048,0.841259,-0.029169,-14.657510,0.000475,0.009463,5.010282
538,d5_t6_0uL_r4,7,12198720,1.742674e+06,0.999327,0.989817,-0.270953,-0.537701,0.998981,0.999448,...,3.638284,0.004374,0.999586,0.628030,0.840835,-0.026932,-13.671820,0.000473,0.007960,3.982672
539,d5_t6_0uL_r5,7,12198720,1.742674e+06,0.999582,0.993589,-0.395606,-0.727265,0.999156,0.999658,...,4.434142,0.003565,0.999553,0.608552,0.823491,-0.028737,-14.495056,0.000482,0.015201,4.815898
540,d5_t6_0uL_r6,7,12198720,1.742674e+06,0.999288,0.995381,-0.217726,-0.664407,0.999305,0.999595,...,13.810241,0.002115,0.998255,0.530583,0.843074,-0.044902,-23.435241,0.000513,0.037457,6.784895


In [47]:
dfC_clean

,identificador,cor_RG,cor_RB,cor_RH,cor_RS,cor_RV,cor_RI,cor_GB,cor_GH,cor_GS,...,com_inertia,com_energy,com_correlation,com_homogeniety,FFT_energy,FFT_entropy,FFT_intertia,FFT_homogeneity,SNM,EME
0,d1_t2_100ul_r1,0.995674,0.989471,-0.383639,-0.130794,0.997174,0.997820,0.993940,-0.348508,-0.140715,...,30.267896,0.001518,0.995921,0.535464,0.882896,-0.039847,9.580998,0.001363,0.147521,4.186365
1,d1_t2_100uL_r2,0.994981,0.989579,0.075174,0.118834,0.996522,0.997343,0.997368,0.138526,0.149192,...,13.167428,0.003691,0.996867,0.516007,0.781416,-0.061916,31.988921,0.000541,0.001729,8.132289
2,d1_t2_100uL_r3,0.995855,0.990799,0.032431,0.017351,0.996876,0.997784,0.997282,0.072029,0.043852,...,12.198123,0.003542,0.997089,0.519729,0.764936,-0.061312,31.311636,0.000535,0.001321,8.409600
3,d1_t2_100uL_r3B,0.997074,0.991002,-0.238737,-0.181805,0.997676,0.998344,0.996343,-0.215537,-0.193286,...,19.078551,0.001026,0.997699,0.396827,0.842077,-0.060000,26.460279,0.000504,0.048926,6.696373
4,d1_t2_100uL_r4,0.996877,0.990841,-0.418375,-0.322490,0.998054,0.998266,0.996252,-0.392721,-0.342047,...,26.928049,0.001044,0.996857,0.392886,0.820254,-0.064946,30.044119,0.000526,0.069647,8.170084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,d5_t6_0uL_r3,0.999056,0.987996,-0.300870,-0.671761,0.999079,0.999271,0.990981,-0.288935,-0.675030,...,4.164741,0.003521,0.999539,0.602048,0.841259,-0.029169,-14.657510,0.000475,0.009463,5.010282
538,d5_t6_0uL_r4,0.999327,0.989817,-0.270953,-0.537701,0.998981,0.999448,0.992072,-0.261745,-0.538167,...,3.638284,0.004374,0.999586,0.628030,0.840835,-0.026932,-13.671820,0.000473,0.007960,3.982672
539,d5_t6_0uL_r5,0.999582,0.993589,-0.395606,-0.727265,0.999156,0.999658,0.994898,-0.386392,-0.720728,...,4.434142,0.003565,0.999553,0.608552,0.823491,-0.028737,-14.495056,0.000482,0.015201,4.815898
540,d5_t6_0uL_r6,0.999288,0.995381,-0.217726,-0.664407,0.999305,0.999595,0.996182,-0.217153,-0.663631,...,13.810241,0.002115,0.998255,0.530583,0.843074,-0.044902,-23.435241,0.000513,0.037457,6.784895


## (função) Remover colunas maior correlação
Parâmetros
- df
- limite

Retorno:
- df: sem as colunas de alta correlação

In [5]:
# Função para remover colunas de um DataFrame que têm uma correlação acima de um limite especificado
def remove_high_correlation_features(df, limite):

    # Selecionar apenas colunas numéricas para o cálculo da correlação
    df_numeric = df.select_dtypes(include=['number'])

    # Criar um conjunto para armazenar os nomes das colunas que têm alta correlação
    correlated_features = set()

    # Calcular a matriz de correlação do DataFrame
    correlation_matrix = df_numeric.corr()

    # Percorrer a matriz de correlação para identificar colunas com alta correlação
    for i in range(len(correlation_matrix.columns)):
        for j in range(i):

            # Verificar se a correlação entre duas colunas é maior que o limite
            if abs(correlation_matrix.iloc[i, j]) > limite:

              # Adicionar o nome da coluna ao conjunto de colunas correlacionadas
                colname = correlation_matrix.columns[i]
                correlated_features.add(colname)

    # Imprimir as colunas que serão removidas para depuração
    print(f"Colunas removidas devido à alta correlação: {correlated_features}")

    # Retornar o DataFrame sem as colunas de alta correlação
    return df.drop(labels=correlated_features, axis=1)


## Criação dos Datasets 
    Prepara os dados para modelagem removendo colunas altamente correlacionadas com base no limite fornecido e adicionando a coluna 'area'.

    Parâmetros:
    - limite_correlacao: valor limite para considerar correlação alta.

    Retorna:
    - dfC_clean_CorrelationFix_mergedWithA: DataFrame com colunas menos correlacionadas e coluna 'area'.


In [6]:
def createDataset(limite_correlacao):
    """
    Prepara os dados para modelagem removendo colunas altamente correlacionadas com base no limite fornecido e adicionando a coluna 'area'.

    Parâmetros:
    - limite_correlacao: valor limite para considerar correlação alta.

    Retorna:
    - dfC_clean_CorrelationFix_mergedWithA: DataFrame com colunas menos correlacionadas e coluna 'area'.
    """
 
    dfC_mergedWithA = pd.merge(dfA, dfC_clean, on='identificador', how='left')
    dfC_clean_CorrelationFix_mergedWithA = remove_high_correlation_features(dfC_mergedWithA, limite_correlacao)
    dfC_clean_CorrelationFix_mergedWithA.rename(columns={'Area(mm²)': 'area'}, inplace=True)
    return dfC_clean_CorrelationFix_mergedWithA


## Treinamento dos modelos

In [7]:
def trainemodel(df_input, model_name):
  # Definindo X como a lista de colunas em df07_merged, excluindo 'identificador' e 'area'
  X_DADOS = list(df_input.drop(['identificador', 'area'], axis=1).columns)  
  # Definindo Y como a variável alvo 'area'
  y = 'area'

  # Removendo linhas que contenham valores faltantes nas colunas de X
  df_input.dropna(subset=X_DADOS, inplace=True)

  # Definindo o k-fold estratificado: dividindo em 5 quantis, 5 folds repetidos 10 vezes
  cv_stratified = RepeatedContinuousStratifiedKFold(n_bins=5, n_splits=5, n_repeats=10, method="quantile", random_state=123)

  # Definindo as métricas de avaliação
  scoring = ["neg_mean_absolute_error", "neg_mean_absolute_percentage_error", "neg_root_mean_squared_error"]

  # Configuração completa para validação cruzada
  scores, model, inspector = run_cross_validation(
      X=X_DADOS,
      y=y,
      data=df_input,

      # Padronização das features pelo Z-score

      preprocess="zscore",
      cv=cv_stratified,
      return_inspector=True,
      problem_type="regression",

      # Algoritmo selecionado para regressão linear robusta (pode ser alterado para "rf" para Random Forest, "svm" para Support Vector Machine, etc.)

      model=model_name,
      return_estimator="all",
      scoring=scoring,
  )
  return scores, model, inspector, X_DADOS

In [8]:
def reportModel(scores):
  # Calcular e imprimir o erro absoluto médio a partir dos resultados da validação cruzada
  print("Erro absoluto médio:", scores["test_neg_mean_absolute_error"].mean() * -1)
  # Calcular e imprimir o erro absoluto médio percentual a partir dos resultados da validação cruzada
  print("Erro absoluto médio em porcentagem:", scores["test_neg_mean_absolute_percentage_error"].mean() * -1)

  # Exibir o resultado de cada simulação em cada fold da validação cruzada
  #scores07

## Importância de cada feature no modelo

In [120]:
def plotar_importancia_caracteristicas(x, modelo, model_name):
    """
    Gera um gráfico de barras horizontais mostrando a importância das características em um modelo linear.

    Parâmetros:
    - modelo: modelo treinado com atributo .coef_ (ex: model["ridge"])
    - titulo: título do gráfico (opcional)
    """

    titulo= "Importância de cada característica no modelo "+model_name 
    
    # Cria um DataFrame 'features' com as características (X07) e suas importâncias no modelo (model07["ridge"].coef_)
    features = pd.DataFrame({"Features": x, "importance": modelo[model_name].coef_})

    # Ordena o DataFrame 'features' pela coluna 'importance' de forma descendente
    features.sort_values(by=["importance"], ascending=True, inplace=True)

    # Adiciona uma coluna 'positive' que indica se a importância é positiva (> 0)
    features["positive"] = features["importance"] > 0

    # Cria uma figura (fig) e um eixo (ax) para o gráfico de barras horizontais, definindo o tamanho da figura como 10x7 polegadas
    fig, ax = plt.subplots(1, 1, figsize=(10, 7))

    # Define o índice do DataFrame 'features' como as características ('Features') para plotagem
    features.set_index("Features", inplace=True)

    # Plota um gráfico de barras horizontais da importância das características ('importance')
    # As barras são coloridas de acordo com a coluna 'positive' (azul para positivo, vermelho para negativo)
    features.importance.plot(
        kind="barh", color=features.positive.map({True: "blue", False: "red"})
    )

    # Define o rótulo do eixo x como "Importancia" e o título do gráfico como "Importancia de cada característica no modelo"
    ax.set(xlabel="Importancia", title=titulo)

    # Exibe o gráfico
    plt.show()


## Teste modelos

In [124]:
def test_model(model_name):
    scores07, model07, inspector09, x07 = trainemodel(df07, model_name)
    scores08, model08, inspector08, x08 = trainemodel(df08, model_name)
    scores09, model09, inspector09, x09 = trainemodel(df09, model_name)

    print("===========================================")
    print("TESTANDO MODELO: "+model_name)
    print("===========================================")
    print("modelo com correlação 0.7")
    print("------------------------------------------")
    reportModel(scores07)
    #plotar_importancia_caracteristicas(x07, model07, model_name)
    print("------------------------------------------")
    print("modelo com correlação 0.8")
    print("------------------------------------------")
    reportModel(scores08)
    #plotar_importancia_caracteristicas(x08, model08, model_name)
    print("------------------------------------------")
    print("modelo com correlação 0.9")
    print("------------------------------------------")
    reportModel(scores09)
    #plotar_importancia_caracteristicas(x09, model09, model_name)
    

In [9]:
df07 = createDataset(0.7)
df08 = createDataset(0.8)
df09 = createDataset(0.9)

Colunas removidas devido à alta correlação: {'Diametro médio', 'cor_SI', 'skew_hist_V', 'com_entropy', 'hu_sobel2', 'lbp_0', 'skew_hist_G', 'com_correlation', 'mean_B', 'cor_GH', 'cor_GS', 'std_hist_B', 'kurt_hist_S', 'hu_canny2', 'lbp_4', 'com_inertia', 'entropy_I', 'lbp_7', 'std_R', 'lbp_1', 'cor_GB', 'std_G', 'lbp_5', 'lbp_8', 'FFT_energy', 'cor_RI', 'skew_hist_R', 'cor_HI', 'cor_BH', 'skew_hist_S', 'kurt_hist_H', 'cor_HV', 'nump_sobel', 'skew_hist_I', 'std_hist_R', 'kurt_hist_I', 'FFT_intertia', 'FFT_homogeneity', 'lbp_9', 'cor_BI', 'lbp_2', 'mean_R', 'skew_hist_B', 'cor_RB', 'mean_I', 'EME', 'cor_VI', 'mean_V', 'kurt_hist_G', 'std_hist_G', 'std_hist_V', 'cor_BS', 'std_B', 'mean_G', 'std_V', 'skew_hist_H', 'kurt_hist_B', 'std_hist_I', 'cor_SV', 'cor_BV', 'com_homogeniety', 'std_I', 'lbp_3', 'com_energy', 'FFT_entropy', 'SNM', 'kurt_hist_R', 'cor_GI'}
Colunas removidas devido à alta correlação: {'Diametro médio', 'cor_SI', 'skew_hist_V', 'com_correlation', 'hu_sobel2', 'lbp_0', 'ske

In [16]:
for col in df09.columns:
    print(col)

identificador
area
cor_RG
cor_RB
cor_RH
cor_RS
cor_RV
cor_GB
cor_GV
cor_GI
cor_BV
cor_HS
cor_VI
mean_H
std_H
mean_S
std_S
mean_V
std_V
std_hist_H
kurt_hist_H
std_hist_S
kurt_hist_S
std_hist_V
kurt_hist_V
kurt_hist_G
nump_sobel
hu_sobel1
hu_sobel3
hu_sobel4
hu_sobel5
hu_sobel6
hu_sobel7
nump_canny
hu_canny1
hu_canny3
hu_canny4
hu_canny5
hu_canny6
hu_canny7
lbp_0
lbp_6
lbp_8
com_entropy
com_energy
FFT_energy
FFT_intertia
FFT_homogeneity
SNM
EME


In [126]:
import warnings

# Ignorar especificamente RuntimeWarnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

## teste modelos regressão

In [127]:
test_model("ridge")

TESTANDO MODELO: ridge
modelo com correlação 0.7
------------------------------------------
Erro absoluto médio: 743.2358644083564
Erro absoluto médio em porcentagem: 5.321160853104791
------------------------------------------
modelo com correlação 0.8
------------------------------------------
Erro absoluto médio: 613.7083981874881
Erro absoluto médio em porcentagem: 4.3428373070159045
------------------------------------------
modelo com correlação 0.9
------------------------------------------
Erro absoluto médio: 591.0066515939366
Erro absoluto médio em porcentagem: 4.043562621181232


In [128]:
test_model("rf")

TESTANDO MODELO: rf
modelo com correlação 0.7
------------------------------------------
Erro absoluto médio: 625.2226121777777
Erro absoluto médio em porcentagem: 3.736354428888947
------------------------------------------
modelo com correlação 0.8
------------------------------------------
Erro absoluto médio: 558.2039572666666
Erro absoluto médio em porcentagem: 3.1452937931302154
------------------------------------------
modelo com correlação 0.9
------------------------------------------
Erro absoluto médio: 521.1368672574074
Erro absoluto médio em porcentagem: 2.689486259821997


In [129]:
test_model("svm")

TESTANDO MODELO: svm
modelo com correlação 0.7
------------------------------------------
Erro absoluto médio: 1279.7661670888272
Erro absoluto médio em porcentagem: 4.4292412499427325
------------------------------------------
modelo com correlação 0.8
------------------------------------------
Erro absoluto médio: 1276.6332407546668
Erro absoluto médio em porcentagem: 4.430450162976828
------------------------------------------
modelo com correlação 0.9
------------------------------------------
Erro absoluto médio: 1275.3877266314105
Erro absoluto médio em porcentagem: 4.4202463715303315


In [139]:
test_model("et")

TESTANDO MODELO: et
modelo com correlação 0.7
------------------------------------------
Erro absoluto médio: 569.849349525926
Erro absoluto médio em porcentagem: 3.602816425841532
------------------------------------------
modelo com correlação 0.8
------------------------------------------
Erro absoluto médio: 510.67318204814813
Erro absoluto médio em porcentagem: 2.885988053249713
------------------------------------------
modelo com correlação 0.9
------------------------------------------
Erro absoluto médio: 469.39127882037036
Erro absoluto médio em porcentagem: 2.448980517598477


In [141]:
test_model("adaboost")

TESTANDO MODELO: adaboost
modelo com correlação 0.7
------------------------------------------
Erro absoluto médio: 816.4120636237974
Erro absoluto médio em porcentagem: 6.290243088185423
------------------------------------------
modelo com correlação 0.8
------------------------------------------
Erro absoluto médio: 701.5098084489331
Erro absoluto médio em porcentagem: 5.453303405595205
------------------------------------------
modelo com correlação 0.9
------------------------------------------
Erro absoluto médio: 622.9095181995885
Erro absoluto médio em porcentagem: 4.464200136602826


In [142]:
test_model("bagging")

TESTANDO MODELO: bagging
modelo com correlação 0.7
------------------------------------------
Erro absoluto médio: 653.8242325185184
Erro absoluto médio em porcentagem: 3.6969121637490043
------------------------------------------
modelo com correlação 0.8
------------------------------------------
Erro absoluto médio: 589.3018555740741
Erro absoluto médio em porcentagem: 3.1999012607290815
------------------------------------------
modelo com correlação 0.9
------------------------------------------
Erro absoluto médio: 548.6861673333334
Erro absoluto médio em porcentagem: 2.6890963997394937


In [145]:
test_model("gauss")

TESTANDO MODELO: gauss
modelo com correlação 0.7
------------------------------------------
Erro absoluto médio: 1163.8305177974883
Erro absoluto médio em porcentagem: 1.2018154270972847
------------------------------------------
modelo com correlação 0.8
------------------------------------------
Erro absoluto médio: 1352.2452140461994
Erro absoluto médio em porcentagem: 0.9079453085227247
------------------------------------------
modelo com correlação 0.9
------------------------------------------
Erro absoluto médio: 1388.770005112338
Erro absoluto médio em porcentagem: 0.9059612320246021


In [147]:
test_model("dummy")

TESTANDO MODELO: dummy
modelo com correlação 0.7
------------------------------------------
Erro absoluto médio: 1387.2651031875857
Erro absoluto médio em porcentagem: 8.928711930913828
------------------------------------------
modelo com correlação 0.8
------------------------------------------
Erro absoluto médio: 1387.2651031875857
Erro absoluto médio em porcentagem: 8.928711930913828
------------------------------------------
modelo com correlação 0.9
------------------------------------------
Erro absoluto médio: 1387.2651031875857
Erro absoluto médio em porcentagem: 8.928711930913828


In [149]:
test_model("ridgecv")

TESTANDO MODELO: ridgecv
modelo com correlação 0.7
------------------------------------------
Erro absoluto médio: 753.6335196225449
Erro absoluto médio em porcentagem: 5.404767705919408
------------------------------------------
modelo com correlação 0.8
------------------------------------------
Erro absoluto médio: 615.2332287659635
Erro absoluto médio em porcentagem: 4.345612852600459
------------------------------------------
modelo com correlação 0.9
------------------------------------------
Erro absoluto médio: 595.9199583152357
Erro absoluto médio em porcentagem: 4.060674233324669


In [150]:
test_model("sgd")

TESTANDO MODELO: sgd
modelo com correlação 0.7
------------------------------------------
Erro absoluto médio: 746.9083595940094
Erro absoluto médio em porcentagem: 5.347128148770297
------------------------------------------
modelo com correlação 0.8
------------------------------------------
Erro absoluto médio: 627.4285821648539
Erro absoluto médio em porcentagem: 4.394155168804707
------------------------------------------
modelo com correlação 0.9
------------------------------------------
Erro absoluto médio: 607.3848858307258
Erro absoluto médio em porcentagem: 4.217855930410768


In [151]:
test_model("linreg")

TESTANDO MODELO: linreg
modelo com correlação 0.7
------------------------------------------
Erro absoluto médio: 742.7733748898079
Erro absoluto médio em porcentagem: 5.31880643886949
------------------------------------------
modelo com correlação 0.8
------------------------------------------
Erro absoluto médio: 614.9485226156762
Erro absoluto médio em porcentagem: 4.356699324989897
------------------------------------------
modelo com correlação 0.9
------------------------------------------
Erro absoluto médio: 590.0983459368057
Erro absoluto médio em porcentagem: 4.040295199371499


In [154]:
test_model("gradientboost")

TESTANDO MODELO: gradientboost
modelo com correlação 0.7
------------------------------------------
Erro absoluto médio: 570.4467825516396
Erro absoluto médio em porcentagem: 3.303577879524005
------------------------------------------
modelo com correlação 0.8
------------------------------------------
Erro absoluto médio: 521.8622704401333
Erro absoluto médio em porcentagem: 2.7889980963810928
------------------------------------------
modelo com correlação 0.9
------------------------------------------
Erro absoluto médio: 496.6214169589902
Erro absoluto médio em porcentagem: 2.492023162237512


In [136]:
julearn.models.list_models()

['svm',
 'rf',
 'et',
 'adaboost',
 'bagging',
 'gradientboost',
 'stacking',
 'gauss',
 'logit',
 'logitcv',
 'linreg',
 'ridge',
 'ridgecv',
 'sgd',
 'nb_bernoulli',
 'nb_categorical',
 'nb_complement',
 'nb_gaussian',
 'nb_multinomial',
 'ds',
 'dummy']